This notebook is used to compile all our models and compare them along with some standardized format. Make sure your code output fits as input to this standardized format, such that it only needs to be imported and can run from the get-go.

I'll also build some plotter to make this work well and of course ensure there is proper docstrings and comments

In [35]:
"""
To import the code right next to the .ipynb file, click on the "Code" parent folder -> add shortcut to drive. And then select "MyDrive"
"""


from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
import sys

os.chdir('/content/drive/MyDrive/Code')
!ls

Mounted at /content/drive
'Code for AI project.ipynb'   Emiel_Robust_PCA.py
 Emiel_Elliptic_Envelope.py   __pycache__


**Emiel's code:**

In [38]:
# Elliptic Envelope:
# !pip install numpy
# !pip install sklearn
# !pip install scipy
# !pip install matplotlib
# !pip install pandas
# !pip install seaborn
import Emiel_Elliptic_Envelope as elliptic_envelope_emiel

# Robust PCA
# !pip install rpca
# !pip install pyrpca
import Emiel_Robust_PCA as rpca_emiel

# print(elliptic_envelope_emiel.main())
print(rpca_emiel.run())


AttributeError: module 'Emiel_Robust_PCA' has no attribute 'run'